In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium

%matplotlib inline
import matplotlib.pyplot as plt

from folium.plugins import FastMarkerCluster # python map data visualization tool

In [83]:
df = pd.read_csv("NBI_Colorado_2019.csv")
list(df.columns.values.tolist())

C:\Users\tobby\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (71,73,102) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['STATE_CODE_001',
 'STRUCTURE_NUMBER_008',
 'RECORD_TYPE_005A',
 'ROUTE_PREFIX_005B',
 'SERVICE_LEVEL_005C',
 'ROUTE_NUMBER_005D',
 'DIRECTION_005E',
 'HIGHWAY_DISTRICT_002',
 'COUNTY_CODE_003',
 'PLACE_CODE_004',
 'FEATURES_DESC_006A',
 'CRITICAL_FACILITY_006B',
 'FACILITY_CARRIED_007',
 'LOCATION_009',
 'MIN_VERT_CLR_010',
 'KILOPOINT_011',
 'BASE_HWY_NETWORK_012',
 'LRS_INV_ROUTE_013A',
 'SUBROUTE_NO_013B',
 'LAT_016',
 'LONG_017',
 'DETOUR_KILOS_019',
 'TOLL_020',
 'MAINTENANCE_021',
 'OWNER_022',
 'FUNCTIONAL_CLASS_026',
 'YEAR_BUILT_027',
 'TRAFFIC_LANES_ON_028A',
 'TRAFFIC_LANES_UND_028B',
 'ADT_029',
 'YEAR_ADT_030',
 'DESIGN_LOAD_031',
 'APPR_WIDTH_MT_032',
 'MEDIAN_CODE_033',
 'DEGREES_SKEW_034',
 'STRUCTURE_FLARED_035',
 'RAILINGS_036A',
 'TRANSITIONS_036B',
 'APPR_RAIL_036C',
 'APPR_RAIL_END_036D',
 'HISTORY_037',
 'NAVIGATION_038',
 'NAV_VERT_CLR_MT_039',
 'NAV_HORR_CLR_MT_040',
 'OPEN_CLOSED_POSTED_041',
 'SERVICE_ON_042A',
 'SERVICE_UND_042B',
 'STRUCTURE_KIND_043A',
 '

In [84]:
df.head()

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,BRIDGE_LEN_IND_112,SCOUR_CRITICAL_113,FUTURE_ADT_114,YEAR_OF_FUTURE_ADT_115,MIN_NAV_CLR_MT_116,FED_AGENCY,SUBMITTED_BY,BRIDGE_CONDITION,LOWEST_RATING,DECK_AREA
0,8,0400-4,1,6,8,50,0,0,85,0,...,Y,5,150,2035,NaN,Y,69,G,7,343.17
1,8,0400-5,1,6,8,0,0,0,85,0,...,Y,6,100,2035,NaN,Y,69,G,7,200.54
2,8,0400-6,1,6,8,0,0,0,85,51385,...,Y,6,30,2035,NaN,Y,69,G,7,376.98
3,8,0403-1,1,6,8,330,0,0,77,0,...,Y,8,200,2035,NaN,Y,69,G,8,241.98
4,8,0403-2,1,6,8,330,0,0,77,0,...,Y,8,100,2035,NaN,Y,69,G,7,267.50


In [85]:
df.describe()

,STATE_CODE_001,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,CRITICAL_FACILITY_006B,MIN_VERT_CLR_010,...,YEAR_RECONSTRUCTED_106,PERCENT_ADT_TRUCK_109,NATIONAL_NETWORK_110,PIER_PROTECTION_111,FUTURE_ADT_114,YEAR_OF_FUTURE_ADT_115,MIN_NAV_CLR_MT_116,SUBMITTED_BY,LOWEST_RATING,DECK_AREA
count,8785.0,8785.0,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,8785.000000,0.0,8785.000000,...,8747.000000,8559.000000,8785.000000,114.000000,8785.000000,8785.000000,8653.0,8785.000000,8785.000000,8785.000000
mean,8.0,1.0,3.573022,1.278088,0.003984,29.964599,58.225498,10170.248150,NaN,94.783267,...,187.362296,6.984811,0.290723,1.035088,12116.190438,2033.516904,0.0,10.048378,6.150370,575.923607
std,0.0,0.0,1.469925,1.346149,0.099441,12.937358,36.427499,19969.143522,NaN,19.154547,...,581.039226,8.261642,0.454122,0.374634,26464.314100,3.741014,0.0,10.788961,0.974186,1214.727470
min,8.0,1.0,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,NaN,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1995.000000,0.0,8.000000,0.000000,24.400000
25%,8.0,1.0,2.000000,1.000000,0.000000,19.000000,31.000000,0.000000,NaN,99.990000,...,0.000000,2.000000,0.000000,1.000000,272.000000,2031.000000,0.0,8.000000,6.000000,122.610000
50%,8.0,1.0,4.000000,1.000000,0.000000,32.000000,59.000000,0.000000,NaN,99.990000,...,0.000000,5.000000,0.000000,1.000000,2366.000000,2032.000000,0.0,8.000000,6.000000,253.760000
75%,8.0,1.0,4.000000,1.000000,0.000000,41.000000,85.000000,16000.000000,NaN,99.990000,...,0.000000,9.000000,1.000000,1.000000,11716.000000,2037.000000,0.0,8.000000,7.000000,604.560000
max,8.0,1.0,8.000000,8.000000,4.000000,68.000000,125.000000,93000.000000,NaN,99.990000,...,2018.000000,95.000000,1.000000,5.000000,337440.000000,2040.000000,0.0,79.000000,9.000000,52912.540000


In [86]:
df.shape

(8785, 123)

In [88]:
df.info(verbose=True)
df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8785 entries, 0 to 8784
Data columns (total 123 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   STATE_CODE_001           int64  
 1   STRUCTURE_NUMBER_008     object 
 2   RECORD_TYPE_005A         int64  
 3   ROUTE_PREFIX_005B        int64  
 4   SERVICE_LEVEL_005C       int64  
 5   ROUTE_NUMBER_005D        object 
 6   DIRECTION_005E           int64  
 7   HIGHWAY_DISTRICT_002     int64  
 8   COUNTY_CODE_003          int64  
 9   PLACE_CODE_004           int64  
 10  FEATURES_DESC_006A       object 
 11  CRITICAL_FACILITY_006B   float64
 12  FACILITY_CARRIED_007     object 
 13  LOCATION_009             object 
 14  MIN_VERT_CLR_010         float64
 15  KILOPOINT_011            float64
 16  BASE_HWY_NETWORK_012     float64
 17  LRS_INV_ROUTE_013A       object 
 18  SUBROUTE_NO_013B         float64
 19  LAT_016                  int64  
 20  LONG_017                 int64  
 21  DETOUR_KILOS_

STATE_CODE_001            int64
STRUCTURE_NUMBER_008     object
RECORD_TYPE_005A          int64
ROUTE_PREFIX_005B         int64
SERVICE_LEVEL_005C        int64
                         ...   
FED_AGENCY               object
SUBMITTED_BY              int64
BRIDGE_CONDITION         object
LOWEST_RATING             int64
DECK_AREA               float64
Length: 123, dtype: object

# Method to split DMS string into decimal, minutes, and seconds elements

In [4]:
# print(str(df["LONG_017"][0]))
# seconds = str(df["LONG_017"][0])[-4:]
# minutes = str(df["LONG_017"][0])[3:5]
# degrees = str(df["LONG_017"][0])[:3]
# print("degrees: " + degrees)
# print("minutes: " + minutes)
# print("seconds: " + seconds)

print(str(df["LAT_016"][0]))

original_dms = str(df["LAT_016"][0])
temp_dms_periods = ""
temp_dms = ""


minutes = str(df["LAT_016"][0])[-6:-4]
seconds = str(df["LAT_016"][0])[-4:]
seconds_periods = str(df["LAT_016"][0])[-4:-2] + "." + str(df["LAT_016"][0])[-2:]

temp_dms = minutes
temp_dms = temp_dms + seconds

if temp_dms in original_dms:
    degrees = original_dms.replace(temp_dms,'')

temp_dms = degrees
temp_dms = temp_dms + minutes
temp_dms = temp_dms + seconds


temp_dms_periods = degrees
temp_dms_periods = temp_dms_periods + "." + minutes
temp_dms_periods = temp_dms_periods + "." + seconds
    
# print("new_str: " + new_str)
print("temp_dms: " + temp_dms)
print("temp_dms_periods: " + temp_dms_periods)

print("degrees: " + degrees)
print("minutes: " + minutes)
print("seconds: " + seconds)

38263100
temp_dms: 38263100
temp_dms_periods: 38.26.3100
degrees: 38
minutes: 26
seconds: 3100


# Method to convert DMS to decimal coordinates, will need to do 360 - second coordinate to convert form West to East Longitude

In [5]:
def dms2dd(s):

    minutes = s[-6:-4]
    seconds = s[-4:]
    seconds_decimal = s[-4:-2] + "." +s[-2:]

    temp_dms = minutes
    temp_dms = temp_dms + seconds
    
    if temp_dms in s:
        degrees = s.replace(temp_dms,'')
    
    dd = float(degrees) + float(minutes)/60 + float(seconds_decimal)/(60*60);
    
    return dd
    
dms2dd(str(df["LONG_017"][0]))
dms2dd(str(df["LAT_016"][0]))

38.44194444444444

In [6]:
decimal_lats = []
decimal_longs = []

for index, row in df.iterrows():
    str_lat = str(row["LAT_016"])
    str_long = str(row["LONG_017"])
    
    if len(str_lat) < 8 or len(str_long) < 8:
        continue
    
    decimal_lat = dms2dd(str_lat)
    decimal_long = dms2dd(str_long)
    
    decimal_long = 360 - decimal_long
    
    decimal_lats.append(decimal_lat)
    decimal_longs.append(decimal_long)
    
decimal_lats
decimal_longs

df_new = pd.DataFrame()
df_new['decimal_lats'] = decimal_lats
df_new['decimal_longs'] = decimal_longs

df_new

,decimal_lats,decimal_longs
0,38.441944,252.444722
1,38.446944,252.444722
2,38.463889,252.450278
3,39.225556,252.189167
4,39.232139,252.186169
...,...,...
8757,39.760000,257.535917
8758,39.616842,257.599158
8759,39.626692,257.599183
8760,39.572217,257.639106


In [7]:
locations = df_new[['decimal_lats', 'decimal_longs']]
locationlist = locations.values.tolist()
locationlist[7]

[37.79356111111111, 254.1589111111111]

In [8]:
map = folium.Map(zoom_start=12)
for point in range(0, len(locationlist[:100])):
    folium.Marker(locationlist[point]).add_to(map)
map

# Obtain addresses for every coordinate in order to extract data from Climate Data Online Search

In [77]:
import geopy
from geopy.exc import GeocoderTimedOut

In [78]:
def get_zipcode(df, geolocator, lat_field, long_field):
    try:
        location = geolocator.reverse((df[lat_field], df[long_field]))
        return location.raw['address']['postcode']
    except:
        return None
    
def get_address(df, geolocator, lat_field, long_field):
    try:
        location = geolocator.reverse((df[lat_field], df[long_field]))
        return location.raw['address']
    except:
        return None

geolocator = geopy.Nominatim(user_agent='my-application') # initialize geolocator

In [79]:
df

,Lat,Lon
0,29.39291,-98.50925
1,29.39923,-98.51256
2,29.40147,-98.51123
3,29.38752,-98.52372
4,29.39291,-98.50925
5,29.39537,-98.50402
6,29.39343,-98.49707
7,29.39291,-98.50925
8,29.39556,-98.53148


In [80]:
df_new

,decimal_lats,decimal_longs
0,38.441944,252.444722
1,38.446944,252.444722
2,38.463889,252.450278
3,39.225556,252.189167
4,39.232139,252.186169
...,...,...
8757,39.760000,257.535917
8758,39.616842,257.599158
8759,39.626692,257.599183
8760,39.572217,257.639106


# Can uncomment out below if zipcodes need to be extracted from coordinates of NBI data

In [14]:
# zipcodes = df_new.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='decimal_lats', long_field='decimal_longs')

In [15]:
# zipcodes[0]

In [16]:
# zipcodes

# Can uncomment out below if entire address objects need to be extracted from coordinates of NBI data

In [45]:
# addresses = df_new.apply(get_address, axis=1, geolocator=geolocator, lat_field='decimal_lats', long_field='decimal_longs')

In [47]:
# addresses.to_csv('NBI_Colorado_addresses.csv')

In [64]:
# addresses[69]

{'road': 'Lane 1 N',
 'county': 'Alamosa County',
 'state': 'Colorado',
 'country': 'United States of America',
 'country_code': 'us'}

# The below cell appends each dictionary element from list of address dictionaries into separate lists to be later used to construct a dataframe

In [65]:
road_list = []
hamlet_list = []
county_list = []
state_list = []
postcode_list = []
country_list = []
country_code_list = []

for index in range(len(addresses)):
    counter = 0
    
    try:
        road_list.append(addresses[index]['road'])
    except:
        road_list.append(None)
        
    try:
        hamlet_list.append(addresses[index]['hamlet'])
    except:
        hamlet_list.append(None)
        
    try:
        county_list.append(addresses[index]['county'])
    except:
        county_list.append(None)
        
    try:
        state_list.append(addresses[index]['state'])
    except:
        state_list.append(None)
        
    try:
        postcode_list.append(addresses[index]['postcode'])
    except:
        postcode_list.append(None)
        
    try:
        country_list.append(addresses[index]['country'])
    except:
        country_list.append(None)
        
    try:
        country_code_list.append(addresses[index]['country_code'])
    except:
        country_code_list.append(None)

In [70]:
address_df = pd.DataFrame()

address_df['road'] = road_list
address_df['hamlet'] = hamlet_list
address_df['county'] = county_list
address_df['state'] = state_list
address_df['postcode'] = postcode_list
address_df['country'] = country_list
address_df['country_code'] = country_code_list

address_df

,road,hamlet,county,state,postcode,country,country_code
0,Q83 RD,Cimarron,Montrose County,Colorado,81220,United States of America,us
1,Q83 RD,Cimarron,Montrose County,Colorado,81220,United States of America,us
2,None,None,Montrose County,Colorado,None,United States of America,us
3,64 6/10 Road,Heiberger,Mesa County,Colorado,None,United States of America,us
4,64 6/10 Road,Heiberger,Mesa County,Colorado,None,United States of America,us
...,...,...,...,...,...,...,...
8757,County Road U,None,Yuma County,Colorado,None,United States of America,us
8758,County Road X,None,Yuma County,Colorado,80824,United States of America,us
8759,County Road X,None,Yuma County,Colorado,80824,United States of America,us
8760,County Road Y,None,Yuma County,Colorado,80807,United States of America,us


In [72]:
#address_df.to_csv('NBI_Colorado_addresses.csv')

# 70 unique counties compared to 6000+ unique zip codes may be easier to derive climate data based on counties

In [76]:
address_df.county.unique()

array(['Montrose County', 'Mesa County', 'Garfield County',
       'Alamosa County', 'Saguache County', 'Montezuma County',
       'La Plata County', 'Dolores County', 'Pueblo County',
       'Summit County', 'Grand County', 'Yuma County', 'Archuleta County',
       'Jefferson County', 'Hinsdale County', 'Eagle County',
       'Conejos County', 'El Paso County', 'Rio Blanco County',
       'Gunnison County', 'Adams County', 'Rio Grande County',
       'Delta County', 'San Miguel County', 'Ouray County',
       'Routt County', 'Jackson County', 'Mineral County',
       'Boulder County', 'Larimer County', 'Clear Creek County',
       'Lake County', 'Chaffee County', 'Custer County',
       'Huerfano County', 'Douglas County', 'Park County',
       'San Juan County', 'Pitkin County', None, 'Moffat County',
       'City and County of Broomfield', 'Weld County', 'Laramie County',
       'Logan County', 'Sedgwick County', 'Arapahoe County',
       'Las Animas County', 'Denver County', 'Phill

In [17]:
# zipcodes.to_csv('NBI_Colorado_zipcodes.csv')

In [33]:
#zipcodes = pd.read_csv('NBI_Colorado_zipcodes.csv')

In [34]:
#zipcodes

,Unnamed: 0,0
0,0,81220
1,1,81220
2,2,NaN
3,3,NaN
4,4,NaN
...,...,...
8757,8757,NaN
8758,8758,80824
8759,8759,80824
8760,8760,80807
